# Posthoc Analysis of IMAGEN:
The preliminary results in our IMAGEN paper advocates for a more in-depth understanding of what contributes to the significant performance of the ML models for the three time-points: <br>
<li>Baseline (<b>BL</b>): Age <tr> <b>14</b></li>
<li>Follow 1 year (<b>FU1</b>): Age <b>16</b></li>
<li>Follow 2 year (<b>FU2</b>): Age <b>19</b></li>
<li>Follow 3 year (<b>FU3</b>): Age <b>22</b></li>

<br>
Such in-depth understanding can be achieved by performing follow-up analysis:

1. Summary statistics
2. Sensitivity analysis
3. Error analysis
4. Visualization SHAP

# 4. Visualization SHAP
## 4.1. SHAP values
 1. what is the best/fastest estimator for my 4 models?
 2. how to save and load SHAP values?
 3. What to visualize?

In [ ]:
!python imagen_shap_visualization.py

Trying to unpickle estimator VarianceThreshold from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
Trying to unpickle estimator SVC from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
Trying to unpickle estimator VarianceThreshold from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
Trying to unpickle estimator LogisticRegression from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
gener

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd 
import numpy as np
from glob import glob
from os.path import join 
import os 
from scikits.bootstrap import ci
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

from plotResults import *

In [ ]:
#################################################################################
models_dir = sorted(glob("results/newlbls-fu3-espad-fu3-19a-binge-*/*/"))[-1]
models_dir

In [ ]:
import shap

In [ ]:
from joblib import load 

models = {}
model_names = list(set([f.split("_")[0] for f in os.listdir(models_dir) if f.split(".")[-1]=="model"]))

In [ ]:
model_names

In [ ]:
# load all trained models
from joblib import load 

models = {}
model_names = list(set([f.split("_")[0] for f in os.listdir(models_dir) if f.split(".")[-1]=="model"]))
for model_name in model_names:
    models.update({model_name: [load(f) for f in glob(models_dir+f"/{model_name}_*.model")]})

models['LR'][0]

In [ ]:
# load the training data
import h5py

h5_dir = "/ritter/share/data/IMAGEN/h5files/" + models_dir.split("/")[-3] + ".h5"
data = h5py.File(h5_dir, 'r')
data.keys(), data.attrs.keys()

In [ ]:
X = data['X'][()]
y = data[data.attrs['labels'][0]][()]
X_col_names = data.attrs['X_col_names'][()]

X100 = shap.utils.sample(X, 100) # 100 instances for use as the background distribution

X.shape, y.shape, len(X_col_names)

In [ ]:
# load the holdout data


In [ ]:
shap_values_stored = {}

for model_name in models:
    print("generating SHAP plots for model = {} ..".format(model_name))
    if "GB" not in model_name:
        print("skipping model {} as it takes too long".format(model_name))
        continue
    for i, model in enumerate(models[model_name]):
        print(model)
        explainer = shap.Explainer(model.predict, X100, output_names=["Healthy","AUD-risk"])
        shap_values = explainer(X)
        shap_values_stored.update({model_name+str(i): shap_values})
#         if ('model_LR' in model.steps):
#             # plot the coefs of Logistic Regression
#             coefs = {}
#             for i in range(model['model_LR'].coef_.shape[-1]):
#                 coefs.update({X_col_names[i] : model['model_LR'].coef_[0,i].round(4)})
#             pd.Series(coefs).sort_values(key=np.abs)[-15:].plot.barh(title="Model coefficients:")
#             plt.show()

In [ ]:
for k in shap_values_stored:
    shap_values = shap_values_stored[k]
    plt.title(k)
    shap.summary_plot(shap_values, features=X, feature_names=X_col_names, plot_type="bar")
    plt.show()
    shap.summary_plot(shap_values, features=X, feature_names=X_col_names, plot_type="dot")
    plt.show() #plt.savefig("viz/{}_{}_dot.pdf".format(model_name,str(i)))
    shap.group_difference_plot(shap_values.values, group_mask=data['sex'][()].astype(bool), 
                       feature_names=X_col_names, max_display=10)
    plt.show() #plt.savefig("viz/{}_{}_sexdiff.pdf".format(model_name,str(i)))

In [ ]:
model = models['GB'][0]

In [ ]:
# compute the SHAP values for the linear model
explainer = shap.Explainer(model.predict, X100, output_names=["Healthy","AUD-risk"])

In [ ]:
shap_values = explainer(X)

In [ ]:
# for i, sub in enumerate(shap_values): #.values.shape
#     print("sub idx={}".format(i))
#     for j, feature in enumerate(sub):
#         print("feature idx={}".format(j))
#         display(type(feature), feature)
        

# shap_values[...,0]

In [ ]:
# clustering = shap.utils.hclust(X, y)

In [ ]:
shap.summary_plot(shap_values, features=X, feature_names=X_col_names, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values, features=X, feature_names=X_col_names, plot_type="dot")

In [ ]:
shap.group_difference_plot(shap_values.values, group_mask=data['sex'][()].astype(bool), 
                           feature_names=X_col_names, max_display=10)

In [ ]:
# individual features
# shap.embedding_plot(396, shap_values.values, feature_names=X_col_names)
# shap.dependence_plot(ind, shap_values=None, features=None, feature_names=None, display_features=None)
# individual subject predictions
#  shap.waterfall_plot(shap_values, max_display=10, show=True)

In [ ]:
model = models['LR'][0]
# compute the SHAP values for the linear model
explainer = shap.Explainer(model.predict, X100, output_names=["Healthy","AUD-risk"], feature_names=X_col_names)

In [ ]:
shap_values_lr = explainer(X)

In [ ]:
# plot the coefs of Logistic Regression
coefs = {}

for i in range(model['model_LR'].coef_.shape[-1]):
    coefs.update({X_col_names[i] : model['model_LR'].coef_[0,i].round(4)})
pd.Series(coefs).sort_values(key=np.abs)[-15:].plot.barh(title="Model coefficients:")
plt.show()

In [ ]:
shap.summary_plot(shap_values_lr, features=X, feature_names=X_col_names, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values_lr, features=X, feature_names=X_col_names, plot_type="dot")

In [ ]:
shap.group_difference_plot(shap_values_lr.values, group_mask=data['sex'][()].astype(bool), 
                           feature_names=X_col_names, max_display=10)

In [ ]:
model = models['SVM-rbf'][0]
# compute the SHAP values for the linear model
explainer = shap.Explainer(model.predict, X100, output_names=["Healthy","AUD-risk"], feature_names=X_col_names)

In [ ]:
shap_values_svmrbf = explainer(X)

In [ ]:
shap.summary_plot(shap_values_svmrbf, features=X, feature_names=X_col_names, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values_svmrbf, features=X, feature_names=X_col_names, plot_type="dot")

In [ ]:
shap.group_difference_plot(shap_values_svmrbf.values, group_mask=data['sex'][()].astype(bool), 
                           feature_names=X_col_names, max_display=10)

In [ ]:
model = models['SVM-lin'][0]
# compute the SHAP values for the linear model
explainer = shap.Explainer(model.predict, X100, output_names=["Healthy","AUD-risk"], feature_names=X_col_names)
shap_values_svmlin = explainer(X)

In [ ]:
shap.summary_plot(shap_values_svmlin, features=X, feature_names=X_col_names, plot_type="bar")
shap.summary_plot(shap_values_svmlin, features=X, feature_names=X_col_names, plot_type="dot")
shap.group_difference_plot(shap_values_svmlin.values, group_mask=data['sex'][()].astype(bool), 
                           feature_names=X_col_names, max_display=10)